In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Transfer Learning with TensorFlow: Feature Extraction**

To improve models we can use several tenchniques like
- Adding More Layers
- Changing the Learning Rate
- Adjusting the Number of Neurons Per Layer

However, instead of above we can use **Transfer Learning**.
- **Transfer Learning** is taking the patterns (also called weights) from another model and applying it on new problem.

Two main benefits of using Transfer Learning.
- Leverage existing Neural Network Architecture proven to work on problems similar to our own
- Using **Already Learned Patterns** on similar data to our own

So, instead of hand-crafting our own **Neural Network Architecture or building them from scratch** we can utilize models which have worked for others.

We can take the patterns a model has learned from datasets such as **ImageNet** and use it as a foundational model.

**What we will Learn**
- Use a smaller dataset to experiment faster (10% of training samples of 10 classes of food)
- Build a Transfer Learning Feature Extraction model using **TensorFlow Hub**
- Introduce a TesnorBoard Callback to track model training results

**Transfer Learning with TensorFlow Hub: Getting great results with only 10% of data**

- **TensorFlow Hub:-** is a repository for existing model components. You can import and use a **Fully Trained Model** using a *URL*

Using the **Pre-trained Models** we can get the results of a fully trained model with only 10% of data.

**Transfer Learning often allows you to get great results with less data**


Let's download 10% of training data from **10_food_classes** dataset and use it to train a food image classifier on it.















In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-09-17 07:31:17--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.207, 172.217.204.207, 172.217.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   152MB/s    in 1.1s    

2025-09-17 07:31:18 (152 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
import zipfile
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/tra

**Creating Data Loaders (Preparing the Data)**

- Create the **ImageDataGenerator** class using the **flow_from_directory** method to load in our images.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")


print("Testing Images")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")



Training Images
Found 750 images belonging to 10 classes.
Testing Images
Found 2500 images belonging to 10 classes.


**Setting Up Callbacks (Things to Run While our Model Trains)**

**Callbacks** are extra functionality that you can add to your models to be performed during or after training. Some of the most important callbacks are

- **Experiment Tracking with TensorBoard:-** Log the performance of multiple models and then view and compare these models in a visual way on **TensorBoard**. **TensorBoard** is a dashboard for inspecting **Neural Network Parameters**

- **Model CheckPointing:-** Save your model as you train so that you can stop training if needed and continue off where you left. It is helpful if training takes a long time and cannot be done in one sitting.

- **Early Stopping:-** Leave your model training for a arbitary amount of time and have it stop training automatically when it ceases to improve. It is helpful when you have a large dataset and do not know how long training will take.

- The TensorBoard Callback can be accessed using
**tf.keras.callbacks.TensorBoard()**. - The main function of this is saving model's training performance metrics to a specified **log_dir**.

- By default, logs are recorded every epoch using the **update_freq='epoch'** parameter. This is a good default but can slow down **Model Training.**





In [7]:
# Creating a tensorboard callback

import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


- We will save the **Model** to a directory [dir_name] / [experiment_name] / [current_timestamp] where
- **dir_name:-** is the overall logs directory
- **experiment_name:-** is the particular experiment
- **current_timestamp:-** is the time the experiment started based on Python time *datatime.datetime().now()